In [1]:
from Bio import SeqIO
import pandas as pd
from Bio.Seq import Seq
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
os.chdir("/home/chensa/gb_data/data_for_analysis/")

In [4]:
paths = ['smprot', 'spencer', 'ahigh']
prefix_path = "/home/chensa/gb_data/data_for_analysis/long/"
id_list = []
lens = []
for path in paths:
    num = 0
    id2all = pickle.load(open(path + f"/{path}_id2all.pkl", "rb"))
    for id,info in id2all.items():
        if type(info['start'])==int:
            num += 1
        else:
            num += len(info['start'])
    print(num)

45
7137
1649


In [90]:
id2all = pickle.load(open("/home/chensa/gb_data/data_for_analysis/ncRNA/ncRNA_id2all.pkl", "rb"))
# with open("ncRNA_5utr_d50.fasta","w") as f:
nc2seq = {}
for id,v in id2all.items():
    regions = v['orf']
    seq = v['rna']
    for start,end in regions:
        nc2seq[id+'_'+str(start)] = seq[:start+4]

### mRNA

In [70]:
id2all = pickle.load(open("/home/chensa/gb_data/data_for_analysis/mRNA/mRNA_id2all.pkl", "rb"))
with open("/home/chensa/gb_data/data_for_analysis/mRNA_d50.fasta","w") as f1:
    with open("/home/chensa/gb_data/data_for_analysis/mRNA_5utr_d1.fasta","w") as f2:
        with open("/home/chensa/gb_data/data_for_analysis/mRNA_orf.fasta","w") as f3:
            with open("/home/chensa/gb_data/data_for_analysis/mRNA_Transcript.fasta","w") as f4:
                for id,v in id2all.items():
                    seq = v['rna']
                    start = v['start']
                    orf = v['aa']
                    # if start+53>20000:continue
                    if start+4<10:continue
                    f4.write('>'+id+'\n'+seq+'\n')
                    f3.write('>'+id+'\n'+orf+'\n')
                    d4 = seq[:start+4];f2.write(">%s_%s\n%s\n" % (id,start,d4))
                    d50 = seq[start+3:start+53];f1.write(">%s_%s\n%s\n" % (id,start,d50))
                    if seq[start:start+3] != 'ATG':
                        print(id,seq[start:start+3])

### ribo_ncRNA

In [65]:
long_list = ["SPENT000505","SPENT001022","SPENT001121","SPENT001152","SPENT001236","SPENT001242","SPENT001245","SPENT001490","SPENT001497","SPENT001628","SPENT002329","SPENT002355","SPENT002479","SPENT002482","SPENT002583","SPENT002812","SPENT003357","SPENT003813","SPENT004029","SPENT004752","SPENT004779","SPENT004824","SPENT004899","SPENT005094","SPENT005273","SPENT005586","SPENT005984","SPENT006710","SPENT006798","ENCT00000134037","ENCT00000173097","ENCT00000478327","MICT00000098400","MICT00000119320"]

In [69]:
paths = ['smprot', 'spencer', 'ahigh']
prefix_path = "/home/chensa/gb_data/data_for_analysis/long/"
id_list = []
lens = []
with open(prefix_path + "ribo_5utr_d1.fasta","w") as f1:
    with open(prefix_path + "ribo_orf.fasta","w") as f2:
        with open(prefix_path + "ribo_d50.fasta","w") as f3:
            with open(prefix_path + 'ribo_Transcript.fasta','w') as w:
                for path in paths:
                    id2all = pickle.load(open(path + f"/{path}_id2all.pkl", "rb"))
                    for id,v in id2all.items():
                        if id not in long_list:continue
                        seq = v['rna']
                        if type(v['start']) == int:
                            start = v['start']
                            if start+4<10:continue
                            lens.append(start+4)
                            d4 = seq[:start+4];f1.write(">%s_%s\n%s\n" % (id,start,d4))
                            orf = v['aa'];f2.write(">%s_%s\n%s\n" % (id,start,orf))
                            d50 = seq[start+3:start+53];f3.write(">%s_%s\n%s\n" % (id,start,d50))
                            if id not in id_list:w.write('>'+id+'\n'+seq+'\n');id_list.append(id)
                        else:
                            for idx,start in enumerate(v['start']):
                                if start+4<10:continue
                                if id not in id_list:w.write('>'+id+'\n'+seq+'\n');id_list.append(id)
                                lens.append(start+4)
                                d4 = seq[:start+4];f1.write(">%s_%s\n%s\n" % (id,start,d4))
                                orf = v['aa'][idx];f2.write(">%s_%s\n%s\n" % (id,start,orf))
                                d50 = seq[start+3:start+53];f3.write(">%s_%s\n%s\n" % (id,start,d50))

! cd-hit -i ribo_Transcript.fasta -o ribo_Transcript.dedup.fasta
! mv ribo_Transcript.fasta ribo_Transcript.fasta.bak
! mv ribo_Transcript.dedup.fasta ribo_Transcript.fasta

In [91]:
path_real = "/home/chensa/gb_data/data_for_analysis/"
os.chdir(path_real)
records = SeqIO.parse(f"{path_real}ribo_Transcript.fasta", "fasta")
keep_list = []
for rec in records:
    keep_list.append(rec.id)
lens = []
records = SeqIO.parse("old_sequences/ribo_5utr_d1.fasta", "fasta")
with open("ribo_5utr_d1.fasta","w") as f:   
    for rec in records:
        tid = rec.id.split('_')[0]
        if tid in keep_list:
            f.write(">%s\n%s\n" % (rec.id, rec.seq))
            lens.append(len(rec.seq))
records = SeqIO.parse("old_sequences/ribo_orf.fasta", "fasta")
with open("ribo_orf.fasta","w") as f:   
    for rec in records:
        tid = rec.id.split('_')[0]
        if tid in keep_list:
            f.write(">%s\n%s\n" % (rec.id, rec.seq))
records = SeqIO.parse("energy/ribo_d50.fasta", "fasta")
with open("ribo_d50.fasta","w") as f:   
    for rec in records:
        tid = rec.id.split('_')[0]
        if tid in keep_list:
            f.write(">%s\n%s\n" % (rec.id, rec.seq))
records = SeqIO.parse("ribo_5utr_d1.fasta", "fasta")
lens = []
for rec in records:
    lens.append(len(rec.seq))

In [92]:
len(lens)

6296

### ncRNA

In [103]:
cnt, len_bin = np.histogram(lens, bins=3000)
cnt, len_bin

(array([1261,  900,  607, ...,    0,    0,    1]),
 array([1.00000000e+01, 1.00960667e+02, 1.91921333e+02, ...,
        2.72710079e+05, 2.72801039e+05, 2.72892000e+05]))

In [94]:
import subprocess
import os
os.chdir("/home/chensa/gb_data/data_for_analysis/")
enst_in_ribo = []
cmd = "grep ENST ribo_Transcript.fasta"
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()
out = out.decode()
out = out.split('\n')
for i in out:
    if i:
        enst_in_ribo.append(i.split('>')[1])

In [104]:
# cmd = "grep ENST tmpres.clstr"
# p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# out, err = p.communicate()
# out = out.decode()
# out = out.split('\n')
# print(len(out))
# for i in out:
#     if i:
#         enst_in_ribo.append(i.split('>')[1].split('_')[0])


bin2id = [[] for _ in range(len(cnt)+1)]
for ncenst, seq in nc2seq.items():
    if ncenst.split('_')[0] in enst_in_ribo:
        continue
    bin2id[np.searchsorted(len_bin, len(seq))-1].append(ncenst)

keep_ids = []
SEED = 1234
np.random.seed(SEED)

for idx,ids in enumerate(bin2id[:-1]):
    sample_cnt = cnt[idx]
    keep_ids.extend(list(np.random.choice(ids, sample_cnt)))

stops = set(["TAA","TAG","TGA"])
with open('ncRNA_Transcript.fasta','w') as w:
    with open("ncRNA_5utr_d1.fasta","w") as f1:
        with open("ncRNA_orf.fasta","w") as f2:
            with open("ncRNA_d50.fasta","w") as f3:
                for orf_id in keep_ids:
                    tid = ('_').join(orf_id.split('_')[:-1])
                    start = int(orf_id.split('_')[-1])
                    rnaseq = id2all[tid]['rna']
                    w.write('>'+tid+'\n'+rnaseq+'\n')
                    d4 = rnaseq[:start+4]
                    f1.write('>'+tid+f'_{str(start)}\n'+d4+'\n')
                    d50 = rnaseq[start+3:start+53]
                    f3.write('>'+tid+f'_{str(start)}\n'+d50+'\n')
                    for i in range(start+3, len(rnaseq), 3):
                        if i+3 > len(rnaseq): break
                        codon = rnaseq[i:i+3]
                        if codon in stops:
                            break
                    orf = rnaseq[start:i+3]
                    f2.write('>'+tid+f'_{str(start)}\n'+orf+'\n')

# cmd = "cd-hit-2d -i ribo_5utr_d1.fasta -i2 ncRNA_5utr_d1.fasta -o tmpres -T 20 -M 100000"
# p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# out, err = p.communicate()

In [96]:
!pwd

/home/chensa/gb_data/data_for_analysis


In [84]:
out.decode().split('\n')

['================================================================',
 'Program: CD-HIT, V4.8.1 (+OpenMP), Jul 28 2024, 08:30:08',
 'Command: cd-hit-2d -i ribo_5utr_d1.fasta -i2',
 '         ncRNA_5utr_d1.fasta -o tmpres -T 20 -M 100000',
 '',
 'Started: Wed Jul 31 15:43:08 2024',
 '================================================================',
 '                            Output                              ',
 '----------------------------------------------------------------',
 'total seq in db1: 6283',
 'total seq in db2: 6296',
 'longest and shortest : 272892 and 11',
 'Total letters: 9722128',
 'Sequences have been sorted',
 'longest and shortest : 271660 and 11',
 'Total letters: 10478897',
 '',
 'Approximated minimal memory consumption:',
 'Sequence        : 21M',
 'Buffer          : 20 X 68M = 1368M',
 'Table           : 2 X 65M = 130M',
 'Miscellaneous   : 0M',
 'Total           : 1520M',
 '',
 'Table limit with the given memory limit:',
 'Max number of representatives: 40

In [47]:
bin2id = [[] for _ in range(len(cnt)+1)]
for ncenst, seq in nc2seq.items():
    if ncenst.split('_')[0] in enst_in_ribo:
        continue
    bin2id[np.searchsorted(len_bin, len(seq))-1].append(ncenst)

keep_ids = []
SEED = 1234
np.random.seed(SEED)

for idx,ids in enumerate(bin2id[:-1]):
    sample_cnt = cnt[idx]
    keep_ids.extend(list(np.random.choice(ids, sample_cnt)))

In [49]:
stops = set(["TAA","TAG","TGA"])
with open('ncRNA_Transcript.fasta','w') as w:
    with open("ncRNA_5utr_d1.fasta","w") as f1:
        with open("ncRNA_orf.fasta","w") as f2:
            with open("ncRNA_d50.fasta","w") as f3:
                for orf_id in keep_ids:
                    tid = ('_').join(orf_id.split('_')[:-1])
                    start = int(orf_id.split('_')[-1])
                    rnaseq = id2all[tid]['rna']
                    w.write('>'+tid+'\n'+rnaseq+'\n')
                    d4 = rnaseq[:start+4]
                    f1.write('>'+tid+f'_{str(start)}\n'+d4+'\n')
                    d50 = rnaseq[start+3:start+53]
                    f3.write('>'+tid+f'_{str(start)}\n'+d50+'\n')
                    for i in range(start+3, len(rnaseq), 3):
                        if i+3 > len(rnaseq): break
                        codon = rnaseq[i:i+3]
                        if codon in stops:
                            break
                    orf = rnaseq[start:i+3]
                    f2.write('>'+tid+f'_{str(start)}\n'+orf+'\n')
    

In [48]:
sum(len(i) for i in bin2id)

2470546